In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 커널
kernel1 = np.ones((2, 2), np.uint8)
kernel2 = np.ones((4, 4), np.uint8)

# 데이터 경로
video_path = './video_data/n_video6.mp4'
video_file = video_path.split('/')[-1]
video_name = video_file.split('.')[0]

cap = cv2.VideoCapture(video_path)

# 프레임 이동
# frame_move = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) // 2)
# cap.set(cv2.CAP_PROP_POS_FRAMES, frame_move)

while cap.isOpened() :
    ret, img = cap.read()
    
    # 다음 프레임 확인 
    if ret is False : # if img is None :
        break
    
    # 이미지 크기 변경
    img = cv2.resize(img, (img.shape[1] //2, img.shape[0]//2),  interpolation=cv2.INTER_AREA)
    
    ######### 1. ROI 영역 추출 #########
    
    # 1) 표지판 ROI
    img_ts = np.zeros(img.shape, img.dtype)
    img_ts[0: img.shape[0]*2//3, :] = img[0: img.shape[0]*2//3, :]
    #cv2.imshow('img_ts', img_ts)
    
    # 2) 도로 ROI
    img_road = np.zeros(img.shape, img.dtype)
    img_road[0: img.shape[0]*2//3, :] = img_road[0: img.shape[0]*2//3, :]
    # cv2.imshow('img_road', img_road)
    
    ######### 2. 붉은색 검출용 마스크 생성 #########
    
    # 색 공간 변환(BGR -> YUV)
    img_ts_yuv = cv2.cvtColor(img_ts, cv2.COLOR_BGR2YUV)
    img_ts_yuv[img_ts_yuv[:, :, 2]<135] = 0
    img_ts_yuv[img_ts_yuv[:, :, 1]<110] = 0
    img_ts_yuv[img_ts_yuv[:, :, 0]>230] = 0
    img_ts_yuv_red = img_ts_yuv[ : , : , 2]
    
    # 이진화
    th, ts_red = cv2.threshold(img_ts_yuv_red, 140, 255, cv2.THRESH_BINARY)
    # 침식
    ts_red_erosion_mask = cv2.erode(ts_red, kernel1, iterations=1)
    # 팽창
    ts_red_dilation_mask = cv2.dilate(ts_red_erosion_mask, kernel2, iterations=1)
    cv2.imshow('ts_red_dilation_mask', ts_red_dilation_mask)
    
    ######### 3. 조명 영향 제거용 마스크 생성 #########
    
    # 캐니 엣지
    ts_edge = cv2.Canny(img_ts, 100, 200)
    
    # 팽창
    ts_edge_dilation_mask = cv2.dilate(ts_edge, kernel1, iterations = 1)
    # 침식
    ts_edge_erosion_mask = cv2.erode(ts_edge_dilation_mask, kernel1, iterations = 1)
    cv2.imshow('ts_edge_erosion_mask', ts_edge_erosion_mask)
    
    ######### 4. 마스크 합성 #########
    ts_mask = cv2.bitwise_and(ts_red_dilation_mask, ts_red_dilation_mask, mask = ts_edge_erosion_mask)
    #cv2.imshow('ts_mask', ts_mask)
    
    ######### 5. 원 검출 #########
    
    # 컨투어 찾기(RETR_EXTERNAL / RETR_TREE )
    _, contours, hierarchy = cv2.findContours(ts_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # 찾은 컨투어 그리기
    ts_contours = np.zeros(ts_mask.shape, ts_mask.dtype)
    cv2.drawContours(ts_contours, contours, -1, 255, 2)
    
    # 객체 찾기
    _, _, stats, centroids = cv2.connectedComponentsWithStats(ts_contours)
    
    for idx, centroid in enumerate(centroids) :
        if stats[idx][0] == 0 and stats[idx][1] == 0 :
            continue
        if np.any(np.isnan(centroid)) :
            continue
            
        x, y, w, h, area = stats[idx]
        centerX, centerY = int(centroid[0]), int(centroid[1])
        
        if area > 200 and area < 1700 and abs(w-h) < 5 :
            try :
                detected_img = img[y-7 : y+h+7,  x-7 : x+w+7]
                gray_detected_img = cv2.cvtColor(detected_img, cv2.COLOR_BGR2GRAY)
                circles = cv2.HoughCircles(gray_detected_img, cv2.HOUGH_GRADIENT, 1, 100, param1=100, param2=45,minRadius=5, maxRadius=38)
                
                if circles is not None :
                    circles = np.uint16(np.around(circles))
                    for i in circles[0, :] :
                        cv2.circle(img, (centerX, centerY), 2, (0, 255, 0), 2)
                        cv2.rectangle(img, (x-7, y-7), (x+w+7, y+h+7), (0, 0, 255), 2)
            except :
                continue
                
    cv2.imshow(video_name, img)
    
    if cv2.waitKey(1) == 27 :
        cap.release()
        cv2.destroyAllWindows()

cap.release()
cv2.destroyAllWindows()

### 표지판 검출 함수화